# Fine-Tuning


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Constants
DATA_PATH = '../data/dataset.xlsx'
MODEL_NAME = "Thammarak/wangchanBERTa-QA-thaiqa_squad"

## Preparing data

In [ ]:
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

# Read data
df = pd.read_excel(DATA_PATH, sheet_name='Default')

# Function to find answer start
def find_answer_start(context, answer):
    return context.find(answer)

# Add context column
df['context'] = df['Question'] + "          " + df['Answer']

# Rename columns
df.rename(columns={'Answer': 'answers', 'Question': 'question'}, inplace=True)

# Split dataset into train, valid, test
len_test = int(0.2 * len(df))
train, valid, test = df[:-len_test], df[-len_test:-len_test*2], df[-len_test*2:]

# Generate question and article IDs
def generate_ids(dataset, offset):
    return [str(i + offset) for i in range(len(dataset))]

train_question_id = generate_ids(train, 0)
train_article_id = generate_ids(train, 0)
valid_question_id = generate_ids(valid, len(train))
valid_article_id = generate_ids(valid, len(train))
test_question_id = generate_ids(test, len(train) + len(valid))
test_article_id = generate_ids(test, len(train) + len(valid))

# Create Dataset objects
train_set = {
    'question_id': train_question_id,
    'article_id': train_article_id,
    'context': train['context'],
    'question': train['question'],
    'answers': train['answers']
}
valid_set = {
    'question_id': valid_question_id,
    'article_id': valid_article_id,
    'context': valid['context'],
    'question': valid['question'],
    'answers': valid['answers']
}
test_set = {
    'question_id': test_question_id,
    'article_id': test_article_id,
    'context': test['context'],
    'question': test['question'],
    'answers': test['answers']
}

# Create DatasetDict
raw_datasets = DatasetDict({
    'train': Dataset.from_dict(train_set),
    'validation': Dataset.from_dict(valid_set),
    'test': Dataset.from_dict(test_set)
})

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-wiki-20210520-spm-finetune-qa')

# Preprocessing function
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1

        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_positions.append(context_start)
            end_positions.append(context_end)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# Apply preprocessing function
tokenized_raw = raw_datasets.map(
    preprocess_function, batched=True, remove_columns=raw_datasets["train"].column_names
)

## Train the model


https://huggingface.co/docs/transformers/tasks/question_answering

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering, DefaultDataCollator

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME).to(device)
data_collator = DefaultDataCollator()

In [ ]:
%%time
import logging
import numpy as np
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# Set up logging
transformers_logger = logging.getLogger("transformers")

# Define training arguments
training_args = TrainingArguments(
    output_dir="models/wangchan_th_wiki_qa",
    overwrite_output_dir=True,
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model='loss',
    greater_is_better=False,
    evaluation_strategy = "epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=1e-05,
    num_train_epochs=100,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    adam_epsilon=1e-06,
    # push_to_hub=True,
)

# Define train and eval datasets
train_dataset = tokenized_raw['train']
eval_dataset = tokenized_raw['validation']

# Map datasets
train_dataset = train_dataset.map(lambda examples: {'input_ids': torch.tensor(examples['input_ids']).to(device),
                                                  'attention_mask': torch.tensor(examples['attention_mask']).to(device),
                                                  'start_positions': torch.tensor(examples['start_positions']).to(device),
                                                  'end_positions': torch.tensor(examples['end_positions']).to(device)})

eval_dataset = eval_dataset.map(lambda examples: {'input_ids': torch.tensor(examples['input_ids']).to(device),
                                                  'attention_mask': torch.tensor(examples['attention_mask']).to(device),
                                                  'start_positions': torch.tensor(examples['start_positions']).to(device),
                                                  'end_positions': torch.tensor(examples['end_positions']).to(device)})

# Set up Trainer with EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # Early stopping with patience of 3 epochs
)


# Train the model
trainer.train()

# trainer.push_to_hub()